In [1]:
import pandas as pd

df = pd.read_csv('../Data/chess.csv', index_col=1)

In [2]:
df = df.drop(["game_id", "site", "date_played", "round", "black", "white", "black_title", "white_title", "time_control", "utc_date", "utc_time", "move_no_pair", "winner", "loser", "winner_elo", "loser_elo"], axis=1)

In [3]:
df = df.loc[df["result"] != "*"]
res = {
  "1-0" : 1,
  "0-1" : -1,
  "1/2-1/2" : 0,
}
df.replace({"result": res}, inplace=True)

filt = (df["termination"] == "Normal") | (df["termination"] == "Time forfeit")
df = df.loc[filt]
res = {
  "Time forfeit" : 0,
  "Normal": 1,
}
df.replace({"termination": res}, inplace=True)

filt = ~(df["variant"] == "Rated Correspondence")
df = df.loc[filt]
res = {
  "Rated Blitz" : 0,
  "Rated Bullet" : 1,
  "Rated Rapid" : 2,
  "Rated Classical" : 3,
  "Rated UltraBullet" : 4
}
df.replace({"variant": res}, inplace=True)

res = {
  "tournament" : 0,
  "game": 1,
}
df.replace({"event": res}, inplace=True)

def remplazar_letra_eco(name):
  if name[0] == "A":
    return int(name.replace("A", "0"))
  elif name[0] == "B":
    return int(name.replace("B", "1"))
  elif name[0] == "C":
    return int(name.replace("C", "2"))
  elif name[0] == "D":
    return int(name.replace("D", "3"))
  elif name[0] == "E":
    return int(name.replace("E", "4"))
df = df.loc[df["eco"] != "?"]
df["eco"] = df.apply(lambda x: remplazar_letra_eco(x["eco"]), axis=1)

filt = df["white_rating_diff"].notna() & df["black_rating_diff"].notna()
df = df.loc[filt]
df["black_rating_diff"] = df["black_rating_diff"].astype('int32')
df["white_rating_diff"] = df["white_rating_diff"].astype('int32')

df["ply_count"] = df["ply_count"].astype('int32')

# Eliminar partidas casuales
filt = (df["white_rating_diff"] == 0) & (df["black_rating_diff"] == 0)
df = df.loc[~filt]

In [4]:
df

,event,result,white_elo,white_rating_diff,black_elo,black_rating_diff,winner_loser_elo_diff,eco,termination,variant,ply_count
game_order,,,,,,,,,,,
1,0,-1,1696,-10,1696,7,0,200,1,2,118
2,0,1,1839,3,1630,-2,209,280,1,0,25
3,0,1,1982,8,2080,-7,-98,300,1,0,55
4,0,1,2088,6,2096,-6,-8,40,1,0,107
5,1,-1,1720,-5,1722,6,2,220,1,0,98
...,...,...,...,...,...,...,...,...,...,...,...
49701,1,-1,1714,-6,1677,6,-37,140,0,0,70
49702,1,1,1157,5,1079,-4,78,230,1,0,35
49703,1,-1,1216,-4,1263,6,47,246,1,0,20


In [5]:
df.to_csv("./Data/chess_numeric.csv")